# The data used for this project's exploratory data analysis has been sourced from:

1) National Highway Traffic Safety Administration API (US data) (https://crashviewer.nhtsa.dot.gov/CrashAPI). 

## Data cleansing of the National Highway Traffic Safety Administration API (US data):

In [10]:
# importing dependencies
import pandas as pd
import numpy as np
import scipy as st
import matplotlib.pyplot as plt
import requests
import json
import csv 

In [15]:
# Import CSV containing road accident data from the year 2020. 
csv_path = "source_data/Accidents_2020_2020.csv"
Accidents_2020 = pd.read_csv(csv_path)
Accidents_2020

,caseyear,state,st_case,statename,ve_total,ve_forms,pvh_invl,peds,pernotmvit,permvit,...,hosp_mn,hosp_mnname,cf1,cf1name,cf2,cf2name,cf3,cf3name,fatals,drunk_dr
0,2020,1,10001,Alabama,1,1,0,0,0,4,...,99,Unknown EMS Hospital Arrival Time,NaN,NaN,NaN,NaN,NaN,NaN,3,1
1,2020,1,10002,Alabama,4,4,0,0,0,6,...,99,Unknown EMS Hospital Arrival Time,NaN,NaN,NaN,NaN,NaN,NaN,1,0
2,2020,1,10003,Alabama,2,2,0,0,0,2,...,99,Unknown EMS Hospital Arrival Time,NaN,NaN,NaN,NaN,NaN,NaN,1,0
3,2020,1,10004,Alabama,1,1,0,0,0,5,...,99,Unknown EMS Hospital Arrival Time,NaN,NaN,NaN,NaN,NaN,NaN,1,0
4,2020,1,10005,Alabama,1,1,0,0,0,1,...,88,Not Applicable (Not Transported),NaN,NaN,NaN,NaN,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36159,2020,56,560111,Wyoming,2,2,0,0,0,2,...,88,Not Applicable (Not Transported),NaN,NaN,NaN,NaN,NaN,NaN,1,0
36160,2020,56,560112,Wyoming,3,3,0,0,0,8,...,50,50,NaN,NaN,NaN,NaN,NaN,NaN,2,0
36161,2020,56,560113,Wyoming,3,3,0,0,0,9,...,99,Unknown EMS Hospital Arrival Time,NaN,NaN,NaN,NaN,NaN,NaN,1,0
36162,2020,56,560114,Wyoming,2,2,0,1,1,3,...,88,Not Applicable (Not Transported),NaN,NaN,NaN,NaN,NaN,NaN,1,0


In [16]:
# The output DataFrame has lots of redundant columns, the below create a new DataFrames with the columns of focus in this exploration stage. 
Cleaned_df = Accidents_2020[["caseyear", "st_case","statename","persons", "countyname", "monthname", 
                                  "day_week", "hour", "rur_urbname", "routename", "latitude", "longitud", 
                                  "harm_evname","lgt_condname", "weathername", "fatals", "drunk_dr"]]
Cleaned_df

,caseyear,st_case,statename,persons,countyname,monthname,day_week,hour,rur_urbname,routename,latitude,longitud,harm_evname,lgt_condname,weathername,fatals,drunk_dr
0,2020,10001,Alabama,4,ELMORE (51),January,4,2,Rural,County Road,32.433133,-86.094850,Tree (Standing Only),Dark - Not Lighted,Clear,3,1
1,2020,10002,Alabama,6,JEFFERSON (73),January,5,17,Urban,Local Street - Municipality,33.484658,-86.839544,Motor Vehicle In-Transport,Dark - Lighted,Rain,1,0
2,2020,10003,Alabama,2,SHELBY (117),January,5,14,Rural,State Highway,33.299942,-86.369642,Ditch,Daylight,Rain,1,0
3,2020,10004,Alabama,5,CALHOUN (15),January,6,15,Rural,County Road,33.795072,-85.883486,Tree (Standing Only),Daylight,Cloudy,1,0
4,2020,10005,Alabama,1,COOSA (37),January,7,0,Rural,County Road,32.848414,-86.083547,Tree (Standing Only),Dark - Not Lighted,Rain,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36159,2020,560111,Wyoming,2,LARAMIE (21),December,7,16,Rural,County Road,41.211981,-105.123781,Motor Vehicle In-Transport,Dark - Not Lighted,Snow,1,0
36160,2020,560112,Wyoming,8,LINCOLN (23),December,6,18,Rural,U.S. Highway,42.814131,-110.957272,Motor Vehicle In-Transport,Dark - Not Lighted,Clear,2,0
36161,2020,560113,Wyoming,9,FREMONT (13),December,6,17,Rural,State Highway,42.993033,-108.406514,Motor Vehicle In-Transport,Dark - Not Lighted,Snow,1,0
36162,2020,560114,Wyoming,3,LARAMIE (21),December,3,20,Urban,U.S. Highway,41.138478,-104.787133,Motor Vehicle In-Transport,Dark - Not Lighted,Clear,1,0


In [17]:
# Checking the data types of the columns in the Cleaned_df. 
Cleaned_df.dtypes

caseyear          int64
st_case           int64
statename        object
persons           int64
countyname       object
monthname        object
day_week          int64
hour              int64
rur_urbname      object
routename        object
latitude        float64
longitud        float64
harm_evname      object
lgt_condname     object
weathername      object
fatals            int64
drunk_dr          int64
dtype: object

In [18]:
# Renaming the columns in Cleaned_df to be self-explanatory.
Cleaned_df = Cleaned_df.rename(columns={"caseyear":"Year", "st_case": "Case_ID","statename":"State","persons": "Person_Count", 
                                                 "countyname":"County", "monthname": "Month", "day_week":"Day_Name", 
                                                 "hour":"Hour", "rur_urbname": "Rural_Urban", "routename":"Route_Name", 
                                                 "latitude": "Latitude", "longitud":"Longitude", "harm_evname": "Harm_Event",
                                                 "lgt_condname":"Visability", "weathername":"Weather_Condition", "fatals":"Fatality_Count", "drunk_dr":"Drunk_Driver_Count"})
Cleaned_df

,Year,Case_ID,State,Person_Count,County,Month,Day_Name,Hour,Rural_Urban,Route_Name,Latitude,Longitude,Harm_Event,Visability,Weather_Condition,Fatality_Count,Drunk_Driver_Count
0,2020,10001,Alabama,4,ELMORE (51),January,4,2,Rural,County Road,32.433133,-86.094850,Tree (Standing Only),Dark - Not Lighted,Clear,3,1
1,2020,10002,Alabama,6,JEFFERSON (73),January,5,17,Urban,Local Street - Municipality,33.484658,-86.839544,Motor Vehicle In-Transport,Dark - Lighted,Rain,1,0
2,2020,10003,Alabama,2,SHELBY (117),January,5,14,Rural,State Highway,33.299942,-86.369642,Ditch,Daylight,Rain,1,0
3,2020,10004,Alabama,5,CALHOUN (15),January,6,15,Rural,County Road,33.795072,-85.883486,Tree (Standing Only),Daylight,Cloudy,1,0
4,2020,10005,Alabama,1,COOSA (37),January,7,0,Rural,County Road,32.848414,-86.083547,Tree (Standing Only),Dark - Not Lighted,Rain,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36159,2020,560111,Wyoming,2,LARAMIE (21),December,7,16,Rural,County Road,41.211981,-105.123781,Motor Vehicle In-Transport,Dark - Not Lighted,Snow,1,0
36160,2020,560112,Wyoming,8,LINCOLN (23),December,6,18,Rural,U.S. Highway,42.814131,-110.957272,Motor Vehicle In-Transport,Dark - Not Lighted,Clear,2,0
36161,2020,560113,Wyoming,9,FREMONT (13),December,6,17,Rural,State Highway,42.993033,-108.406514,Motor Vehicle In-Transport,Dark - Not Lighted,Snow,1,0
36162,2020,560114,Wyoming,3,LARAMIE (21),December,3,20,Urban,U.S. Highway,41.138478,-104.787133,Motor Vehicle In-Transport,Dark - Not Lighted,Clear,1,0
